In [1]:
!pip install -U selfies

import selfies as sf

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install -U joblib

from joblib import Parallel, delayed

Defaulting to user installation because normal site-packages is not writeable


In [3]:
with open('pubchem-10m.txt') as f:
    smiles_strings = list(map(lambda s: s.rstrip('\n'), f.readlines()))
smiles_strings[:10]

['CN(c1ccccc1)c1ccccc1C(=O)NCC1(O)CCOCC1',
 'CC[NH+](CC)C1CCC([NH2+]C2CC2)(C(=O)[O-])C1',
 'COCC(CNC(=O)c1ccc2c(c1)NC(=O)C2)OC',
 'OCCn1cc(CNc2cccc3c2CCCC3)nn1',
 'CCCCCCc1ccc(C#Cc2ccc(C#CC3=CC=C(CCC)CC3)c(C3CCCCC3)c2)c(F)c1',
 'O=C(NCc1ccc(F)cc1)N1CC=C(c2c[nH]c3ccccc23)CC1',
 'Nc1nc2ccc(F)c(F)c2n1Cc1ccccc1Br',
 'COc1cccc(c2cccc(F)c2C(=O)[O-])c1',
 'COc1ccc(CC(=O)Nc2c(C#N)c(C)c(C)n2Cc2ccccc2)cc1',
 'N#Cc1ccccc1NC(=O)CCSc1nnc(C[NH+]2CCCC2)n1Cc1ccccc1']

In [4]:
from functools import partial

soft_encoder = partial(sf.encoder, strict=False)
selfies_strings = Parallel(n_jobs=-1)(delayed(soft_encoder)(string)
                                      for string in smiles_strings)

In [7]:
with open('pubchem-10m-soft.sf', 'w') as f:
    f.write('\n'.join(selfies_strings))

In [4]:
%%time

strict_selfies_strings = []

def encode_strict(smiles_string):
    try:
        selfies_string = sf.encoder(smiles_string)
        strict_selfies_strings.append(selfies_string)
        return True
    except sf.exceptions.EncoderError:
        return False

encode_mask = Parallel(n_jobs=-1, require='sharedmem')(
    delayed(encode_strict)(string) for string in smiles_strings[:100_000])

assert sum(encode_mask) == len(strict_selfies_strings)

CPU times: user 2min 7s, sys: 6.32 s, total: 2min 14s
Wall time: 2min 8s


In [5]:
len(strict_selfies_strings)

99993